In [2]:
import pandas as pd
from transformers import AutoTokenizer
import random
import json

In [4]:
data = pd.read_csv("../data/ner.csv", encoding='cp1252', on_bad_lines='warn')
data

Skipping line 281837: expected 25 fields, saw 34



Unnamed: 0     lemma next-lemma next-next-lemma next-next-pos  \
0                 0  thousand         of        demonstr           NNS   
1                 1        of   demonstr            have           VBP   
2                 2  demonstr       have           march           VBN   
3                 3      have      march         through            IN   
4                 4     march    through          london           NNP   
...             ...       ...        ...             ...           ...   
1050790     1048570      they    respond              to            TO   
1050791     1048571   respond         to             the            DT   
1050792     1048572        to        the          attack            NN   
1050793     1048573       the     attack            with            IN   
1050794     1048574    attack       with     machine-gun            JJ   

         next-next-shape next-next-word next-pos next-shape      next-word  \
0              lowercase  demonstrators       IN  lowercase             of   
1              lowercase           have      NNS  lowercase  demonstrators   
2              lowercase        marched      VBP  lowercase           have   
3              lowercase        through      VBN  lowercase        marched   
4            capitalized         London       IN  lowercase        through   
...                  ...            ...      ...        ...            ...   
1050790        lowercase             to      VBD  lowercase      responded   
1050791        lowercase            the       TO  lowercase             to   
1050792        lowercase         attack       DT  lowercase            the   
1050793        lowercase           with       NN  lowercase         attack   
1050794  contains-hyphen    machine-gun       IN  lowercase           with   

         ... prev-prev-lemma prev-prev-pos prev-prev-shape prev-prev-word  \
0        ...      __start2__    __START2__        wildcard     __START2__   
1        ...      __start1__    __START1__        wildcard     __START1__   
2        ...        thousand           NNS     capitalized      Thousands   
3        ...              of            IN       lowercase             of   
4        ...        demonstr           NNS       lowercase  demonstrators   
...      ...             ...           ...             ...            ...   
1050790  ...            forc           NNS       lowercase         forces   
1050791  ...            said           VBD       lowercase           said   
1050792  ...            they           PRP       lowercase           they   
1050793  ...         respond           VBD       lowercase      responded   
1050794  ...              to            TO       lowercase             to   

          prev-shape      prev-word sentence_idx        shape           word  \
0           wildcard     __START1__          1.0  capitalized      Thousands   
1        capitalized      Thousands          1.0    lowercase             of   
2          lowercase             of          1.0    lowercase  demonstrators   
3          lowercase  demonstrators          1.0    lowercase           have   
4          lowercase           have          1.0    lowercase        marched   
...              ...            ...          ...          ...            ...   
1050790    lowercase           said      47959.0    lowercase           they   
1050791    lowercase           they      47959.0    lowercase      responded   
1050792    lowercase      responded      47959.0    lowercase             to   
1050793    lowercase             to      47959.0    lowercase            the   
1050794    lowercase            the      47959.0    lowercase         attack   

        tag  
0         O  
1         O  
2         O  
3         O  
4         O  
...      ..  
1050790   O  
1050791   O  
1050792   O  
1050793   O  
1050794   O  

[1050795 rows x 25 columns]

In [5]:
data.sentence_idx.value_counts()

2549.0     140
11994.0    132
608.0      124
5805.0     122
6344.0     120
          ... 
37093.0      2
8412.0       2
39874.0      2
40249.0      2
38917.0      1
Name: sentence_idx, Length: 35177, dtype: int64

In [6]:
sentences = data.sentence_idx.value_counts().index[:20000]
len(sentences)

20000

In [7]:
new_data = data[data['sentence_idx'].isin(sentences)]
new_data.head()

Unnamed: 0     lemma next-lemma next-next-lemma next-next-pos  \
0           0  thousand         of        demonstr           NNS   
1           1        of   demonstr            have           VBP   
2           2  demonstr       have           march           VBN   
3           3      have      march         through            IN   
4           4     march    through          london           NNP   

  next-next-shape next-next-word next-pos next-shape      next-word  ...  \
0       lowercase  demonstrators       IN  lowercase             of  ...   
1       lowercase           have      NNS  lowercase  demonstrators  ...   
2       lowercase        marched      VBP  lowercase           have  ...   
3       lowercase        through      VBN  lowercase        marched  ...   
4     capitalized         London       IN  lowercase        through  ...   

  prev-prev-lemma prev-prev-pos prev-prev-shape prev-prev-word   prev-shape  \
0      __start2__    __START2__        wildcard     __START2__     wildcard   
1      __start1__    __START1__        wildcard     __START1__  capitalized   
2        thousand           NNS     capitalized      Thousands    lowercase   
3              of            IN       lowercase             of    lowercase   
4        demonstr           NNS       lowercase  demonstrators    lowercase   

       prev-word sentence_idx        shape           word tag  
0     __START1__          1.0  capitalized      Thousands   O  
1      Thousands          1.0    lowercase             of   O  
2             of          1.0    lowercase  demonstrators   O  
3  demonstrators          1.0    lowercase           have   O  
4           have          1.0    lowercase        marched   O  

[5 rows x 25 columns]

In [8]:
new_data.tag.value_counts()

O        668694
B-geo     28499
B-org     15345
B-tim     14932
I-per     13417
I-org     13218
B-per     12708
B-gpe     12464
I-geo      5904
I-tim      4765
B-art       348
B-eve       280
I-eve       234
I-art       226
I-gpe       188
B-nat       188
I-nat        65
Name: tag, dtype: int64

In [9]:
labels = new_data.tag.value_counts().index
labels

Index(['O', 'B-geo', 'B-org', 'B-tim', 'I-per', 'I-org', 'B-per', 'B-gpe',
       'I-geo', 'I-tim', 'B-art', 'B-eve', 'I-eve', 'I-art', 'I-gpe', 'B-nat',
       'I-nat'],
      dtype='object')

In [10]:
label_maps = {}
c=0
for k in labels:
    label_maps[k] = c
    c+=1
    
print(label_maps)

{'O': 0, 'B-geo': 1, 'B-org': 2, 'B-tim': 3, 'I-per': 4, 'I-org': 5, 'B-per': 6, 'B-gpe': 7, 'I-geo': 8, 'I-tim': 9, 'B-art': 10, 'B-eve': 11, 'I-eve': 12, 'I-art': 13, 'I-gpe': 14, 'B-nat': 15, 'I-nat': 16}


In [16]:
# save label_map
label_maps = json.dumps(label_maps)
with open("label_maps.json", 'w') as f:
    f.write(label_maps)

In [ ]:
# read back label_maps

In [11]:
# parsing data 
data_json = []
for ind, i in enumerate(sentences):
    ner_tags = []
    tokens = []
    sub_data = new_data[new_data['sentence_idx']==i]
    for j in sub_data.index:
        tokens.append(sub_data.loc[j,'word'])
        ner_tag = label_maps[sub_data.loc[j,'tag']]
        ner_tags.append(ner_tag)
        
    data = {'id':ind, 'ner_tags':ner_tags, 'tokens':tokens}
    data_json.append(data)

In [12]:
print(data_json[0])

{'id': 0, 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'tokens': ['But', 'when', 'the', 'Wolf', ',', 'day', 'after', 'day', ',', 'kept', 'in', 'the', 'company', 'of', 'the', 'sheep', 'and', 'did', 'not', 'make', 'the', 'slightest', 'effort', 'to', 'seize', 'them', ',', 'the', 'Shepherd', 'began', 'to', 'look', 'upon', 'him', 'as', 'a', 'guardian', 'of', 'his', 'flock', 'rather', 'than', 'as', 'a', 'plotter', 'of', 'evil', 'against', 'it', ';', 'and', 'when', 'occasion', 'called', 'him', 'one', 'day', 'into', 'the', 'city', ',', 'he', 'left', 'the', 'sheep', 'entirely', 'in', 'his', 'charge', '.', 'But', 

In [13]:
def train_val_test_split(data, split_size=0.1):
    random.shuffle(data)
    total_size = len(data)
    test_size = val_size = int(total_size*split_size)
    test_indices = random.sample(range(total_size), test_size)
    test_data = [ele for i, ele in enumerate(data) if i in test_indices]
    train_data = [ele for i, ele in enumerate(data) if i not in test_indices]
    total_size = len(train_data)
    val_indices = random.sample(range(total_size), val_size)
    val_data = [ele for i, ele in enumerate(train_data) if i in val_indices]
    train_data = [ele for i, ele in enumerate(train_data) if i not in val_indices]
    return train_data, val_data, test_data

In [14]:
train, val, test = train_val_test_split(data_json, split_size=0.1)
print(train[:3])

[{'id': 11106, 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0], 'tokens': ['The', 'new', 'terms', 'raised', 'prices', 'for', 'above', 'quota', 'purchases', ',', 'increasing', 'Belarus', "'", 'current', 'account', 'deficit', '.', 'The', 'new', 'terms', 'raised', 'prices', 'for', 'above', 'quota', 'purchases', ',', 'increasing', 'Belarus', "'", 'current', 'account', 'deficit', '.']}, {'id': 6325, 'ner_tags': [2, 5, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 5, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0], 'tokens': ['Saudi', 'Arabia', 'has', 'been', 'battling', 'al-Qaida', 'militants', 'in', 'the', 'kingdom', ',', 'and', 'the', 'terrorist', 'group', 'has', 'targeted', 'Westerners', 'in', 'the', 'past', '.', 'Saudi', 'Arabia', 'has', 'been', 'battling', 'al-Qaida', 'militants', 'in', 'the', 'kingdom', ',', 'and', 'the', 'terrorist', 'group', 'has', 'targeted', 'Westerners', 'in', 'the', '

In [15]:
# json_dump
train = json.dumps(train)
val = json.dumps(val)
test = json.dumps(test)

In [19]:
for file,d  in zip(['train', 'val', 'test'], [train, val, test]):
    with open(f"../data/huggingface/{file}.json", 'w') as f:
        f.write(d)

In [9]:
text = "today is a very nice day"

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
w_tokens = text.split()
tokenized_input = tokenizer(w_tokens, is_split_into_words=True)
print(tokenized_input)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

{'input_ids': [101, 2651, 2003, 1037, 2200, 3835, 2154, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'today', 'is', 'a', 'very', 'nice', 'day', '[SEP]']


In [12]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs